In [152]:
# Imports
import os
import h5py
import numpy as np
import random
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, datasets, backend
from tensorflow.keras.layers import Conv2D, Reshape
from tensorflow.keras.callbacks import EarlyStopping

from PIL import Image
from sklearn.model_selection import train_test_split

In [158]:
# Cargar el directorio con los archivos .h5
directory = "./BraTS2020_training_data/content/data"

In [ ]:
# Cargar los archivos en un array
h5_files = [f for f in os.listdir(directory) if f.endswith('.h5')]
print(f"Found {len(h5_files)} .h5 files:\nExample file names:{h5_files[:3]}")

In [ ]:
# Abrir los primeros archivos .h5 de la lista para inspeccionarlos
if h5_files:
    selected_file = random.choice(h5_files)
    file_path = os.path.join(directory, selected_file)
    with h5py.File(file_path, 'r') as file:
        print("\nKeys for each file:", list(file.keys()))
        for key in file.keys():
            print(f"\nData type of {key}:", type(file[key][()]))
            print(f"Shape of {key}:", file[key].shape)
            print(f"Array dtype: {file[key].dtype}")
            print(f"Array max val: {np.max(file[key])}")
            print(f"Array min val: {np.min(file[key])}")
            print("*"*10)
            print(f"Mean: {np.mean(file[key])}")
            print(f"Standard deviation: {np.std(file[key])}")
            print("*"*10)
            
            # Verificar valores nulos
            if np.isnan(file[key]).any():
                print("Hay valores NaN en los datos.")
            else:
                print("No se encontraron valores NaN.")
else:
    print("No .h5 files found.")

In [ ]:
# Método para visualizar los 4 tipos de canales de las IRM
def display_image_channels(image, title='Image Channels'):
    channel_names = ['T1-weighted (T1)', 'T1-weighted post contrast (T1c)', 'T2-weighted (T2)', 'Fluid Attenuated Inversion Recovery (FLAIR)']
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    for idx, ax in enumerate(axes.flatten()):
        channel_image = image[idx, :, :]  # Matriz transpuesta para mostrar el canal
        ax.imshow(channel_image, cmap='magma')    
        ax.axis('off')
        ax.set_title(channel_names[idx])
    plt.tight_layout()
    plt.suptitle(title, fontsize=20, y=1.03)
    plt.show()

In [ ]:
# Método para visualizar las máscaras de las IRM
def display_mask_channels_as_rgb(mask, title='Mask Channels as RGB'):
    channel_names = ['Necrotic (NEC)', 'Edema (ED)', 'Tumour (ET)']
    fig, axes = plt.subplots(1, 3, figsize=(9.75, 5))
    for idx, ax in enumerate(axes):
        rgb_mask = np.zeros((mask.shape[1], mask.shape[2], 3), dtype=np.uint8)
        rgb_mask[..., idx] = mask[idx, :, :] * 255  # Matriz transpuesta para mostrar el canal
        ax.imshow(rgb_mask)
        ax.axis('off')
        ax.set_title(channel_names[idx])
    plt.suptitle(title, fontsize=20, y=0.93)
    plt.tight_layout()
    plt.show()

In [ ]:
# Método para sobreponer las máscaras con las imágenes
def overlay_masks_on_image(image, mask, title='Brain MRI with Tumour Masks Overlay'):
    t1_image = image[0, :, :]  # Usar el primer canal de la imagen
    t1_image_normalized = (t1_image - t1_image.min()) / (t1_image.max() - t1_image.min()) # Normalización de datos

    rgb_image = np.stack([t1_image_normalized] * 3, axis=-1)
    color_mask = np.stack([mask[0, :, :], mask[1, :, :], mask[2, :, :]], axis=-1)
    rgb_image = np.where(color_mask, color_mask, rgb_image)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(rgb_image)
    plt.title(title, fontsize=18, y=1.02)
    plt.axis('off')
    plt.show()

In [ ]:
# Elegir una muestra al azar
sample_selected = random.choice(h5_files)
sample_file_path = os.path.join(directory, sample_selected)
data = {}
with h5py.File(sample_file_path, 'r') as file:
    for key in file.keys():
        data[key] = file[key][()]

In [ ]:
# Transpuesta de la imagen y la máscara para obtener los primeros canales
image = data['image'].transpose(2, 0, 1)
mask = data['mask'].transpose(2, 0, 1)

In [ ]:
# Ver imágenes
display_image_channels(image)
display_mask_channels_as_rgb(mask)
overlay_masks_on_image(image, mask)

In [50]:
# # Función de preprocesamiento de las imágenes
# def preprocess_image(image, mask):
#     # Ajustar píxeles entre 0 y 1 (normalización)
#     image = image.astype('float32')
#     for i in range(image.shape[2]):  # Iterar sobre canales
#         min_val = np.min(image[:, :, i])
#         max_val = np.max(image[:, :, i]) + 1e-4
#         image[:, :, i] = (image[:, :, i] - min_val) / max_val

#     # Convertir la máscara RGB a múltiples canales de clases
#     mask = rgb_to_multi_class_mask(mask)

#     # Máscara tenga la forma (height, width, 3)
#     return image, mask

# # Función para convertir las máscaras RGB a múltiples clases (3 canales)
# def rgb_to_multi_class_mask(mask):
#     mask_class = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    
#     mask_class[np.all(mask == [0, 0, 0], axis=-1)] = [1, 0, 0]  # Necrosis (NEC)
#     mask_class[np.all(mask == [255, 0, 0], axis=-1)] = [0, 1, 0]  # Edema (ED)
#     mask_class[np.all(mask == [0, 255, 0], axis=-1)] = [0, 0, 1]  # Tumor (ET)

#     return mask_class

# # Función para cargar los archivos .h5
# def load_h5_file(file_path):
#     with h5py.File(file_path, 'r') as file:
#         image = file['image'][()]  # Cargar la imagen
#         mask = file['mask'][()]    # Cargar la máscara
        
#         # Preprocesar imagen y máscara
#         return preprocess_image(image, mask)

# # Función para crear el dataset de Brain Scan
# def create_brain_scan_dataset(file_paths):
#     def load_data(file_path):
#         # Decodificar la ruta de archivo
#         file_path = file_path.numpy().decode("utf-8")
        
#         # Leer y procesar el archivo .h5
#         image, mask = load_h5_file(file_path)
        
#         # Convertir a tensores de TensorFlow
#         return tf.convert_to_tensor(image, dtype=tf.float32), tf.convert_to_tensor(mask, dtype=tf.float32)

#     # Crear un dataset de TensorFlow
#     dataset = tf.data.Dataset.from_tensor_slices(file_paths)
    
#     # Mapear la función load_data a cada archivo
#     # dataset = dataset.map(lambda x: tf.py_function(
#     #     load_data, [x], [tf.float32, tf.float32]
#     # ), num_parallel_calls=tf.data.AUTOTUNE)

#     dataset = dataset.map(
#         lambda x: tf.py_function(load_data, [x], [tf.float32, tf.float32]),
#         num_parallel_calls=tf.data.AUTOTUNE
#     )
#     dataset = dataset.map(
#         lambda img, mask: (tf.ensure_shape(img, [None, None, 4]), 
#                            tf.ensure_shape(mask, [None, None, 3])),
#         num_parallel_calls=tf.data.AUTOTUNE
#     )

#     return dataset

# for file_path in h5_files[:5]:  # Revisar los primeros 5 archivos
#     image, mask = load_h5_file(file_path)
#     print("Tamaño de la imagen:", image.shape)
#     print("Tamaño de la máscara:", mask.shape)

# for file_path in h5_files[:5]:
#     with h5py.File(file_path, 'r') as f:
#         print("Claves en el archivo:", list(f.keys()))
#         print("Forma de la imagen:", f['image'].shape)
#         print("Forma de la máscara:", f['mask'].shape)

Tamaño de la imagen: (240, 240, 4)
Tamaño de la máscara: (240, 240, 3)
Tamaño de la imagen: (240, 240, 4)
Tamaño de la máscara: (240, 240, 3)
Tamaño de la imagen: (240, 240, 4)
Tamaño de la máscara: (240, 240, 3)
Tamaño de la imagen: (240, 240, 4)
Tamaño de la máscara: (240, 240, 3)
Tamaño de la imagen: (240, 240, 4)
Tamaño de la máscara: (240, 240, 3)
Claves en el archivo: ['image', 'mask']
Forma de la imagen: (240, 240, 4)
Forma de la máscara: (240, 240, 3)
Claves en el archivo: ['image', 'mask']
Forma de la imagen: (240, 240, 4)
Forma de la máscara: (240, 240, 3)
Claves en el archivo: ['image', 'mask']
Forma de la imagen: (240, 240, 4)
Forma de la máscara: (240, 240, 3)
Claves en el archivo: ['image', 'mask']
Forma de la imagen: (240, 240, 4)
Forma de la máscara: (240, 240, 3)
Claves en el archivo: ['image', 'mask']
Forma de la imagen: (240, 240, 4)
Forma de la máscara: (240, 240, 3)


In [105]:
# # Manipulación de imágenes
# COLOR_MAP = {
#     (0, 0, 0): 0,      # Fondo
#     (255, 0, 0): 1,    # Tumor realzado
#     (0, 255, 0): 2,    # Tumor no realzado/edema
#     (0, 0, 255): 3,    # Tumor núcleo
# }

# # Convertir RGB a clases
# def rgb_to_class(mask):
#     mask_class = np.zeros(mask.shape[:2], dtype=np.uint8)
#     for color, class_idx in COLOR_MAP.items():
#         mask_class[np.all(mask == color, axis=-1)] = class_idx
#     return mask_class

# # Cargar la imagen y la máscara
# def load_image_and_mask(image_path, mask_path):
#     # Cargar la imagen de 4 canales (puedes usar .npy o imágenes dependiendo del formato de tu dataset)
#     image = np.load(image_path)  # (H, W, 4)
#     image = tf.convert_to_tensor(image, dtype=tf.float32)
    
#     # Cargar la máscara RGB
#     mask = np.array(Image.open(mask_path))
    
#     # Convertir la máscara RGB a clases
#     mask = rgb_to_class(mask)
#     mask = tf.convert_to_tensor(mask, dtype=tf.int32)  # (H, W)
    
#     return image, mask

# # TF Dataset
# def preprocess(image_path, mask_path):
#     image, mask = tf.numpy_function(
#         func=load_image_and_mask,
#         inp=[image_path, mask_path],
#         Tout=[tf.float32, tf.int32]
#     )
#     return image, mask

# # Dataset en TensorFlow
# def create_dataset(image_paths, mask_paths, batch_size=8, shuffle=True):
#     dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
#     dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
#     if shuffle:
#         dataset = dataset.shuffle(buffer_size=100)
#     dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
#     return dataset

In [106]:
# # Cargar datos de un archivo H5
# def load_h5_data(file_path):
#     with h5py.File(file_path, 'r') as f:
#         # Ajusta las claves según la estructura del archivo H5
#         images = np.array(f['images'])  # Imágenes de 4 canales
#         masks = np.array(f['masks'])    # Máscaras en formato RGB
#     return images, masks

# # Máscaras
# def process_mask(mask):
#     # Asegúrate de usar la estructura correcta para tus datos
#     mapping = {
#         0: 0,    # Fondo
#         76: 1,   # Edema
#         150: 2,  # Tumor realzado
#         226: 3   # Tumor no realzado
#     }
#     for rgb_value, label in mapping.items():
#         mask = np.where(mask == rgb_value, label, mask)
#     return mask.astype(np.int32)

# # Normalización 
# def normalize_image(image):
#     return (image - np.min(image)) / (np.max(image) - np.min(image))  # Escalar entre 0 y 1

In [143]:
# # Preprocesamiento de entrada
# def load_image(image_path):
#     img = tf.io.read_file(image_path)
#     img = tf.image.decode_png(img, channels=4)  # 4 canales
#     img = tf.image.resize(img, (240, 240))
#     img = tf.cast(img, tf.float32) / 255.0  # Normalización
#     return img

# def load_mask(mask_path):
#     mask = tf.io.read_file(mask_path)
#     mask = tf.image.decode_png(mask, channels=3)
#     mask = tf.image.resize(mask, (240, 240))
#     mask = tf.reduce_max(mask, axis=-1)  # Convertir RGB a índices
#     mask = tf.cast(mask, tf.int32)
#     return mask

# # Dataset
# def preprocess_data(image_path, mask_path):
#     img = load_image(image_path)
#     mask = load_mask(mask_path)
#     return img, mask

# dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
# dataset = dataset.map(preprocess_data)
# dataset = dataset.batch(16).prefetch(tf.data.AUTOTUNE)

NameError: name 'image_paths' is not defined

In [ ]:

# def load_h5(h5_path, dataset_name):
#     with h5py.File(h5_path, 'r') as file:
#         data = file[dataset_name][:]
        
#     return data

# def load_data(directory):
#     image_data = []
#     mask_data = []
#     for file in os.listdir(directory):
#         if file.endswith(".h5"):
#             file = os.path.join(directory, file)

#             # Cargar imágenes y máscaras
#             images = load_h5(file, dataset_name="image")
#             masks = load_h5(file, dataset_name="mask")
            
#             # Agregar a listas
#             image_data.append(images)
#             mask_data.append(masks)

#     # Convertir listas a arrays de NumPy
#     image_data = np.concatenate(image_data, axis=0)
#     mask_data = np.concatenate(mask_data, axis=0)

#     return image_data, mask_data


# # def load_h5_file(h5_path, dataset_name):
# #     with h5py.File(h5_path, 'r') as f:
# #         data = f[dataset_name][:]
# #     return data

# # def load_data(directory_path, image_dataset_name="images", mask_dataset_name="masks"):
# #     image_data = []
# #     mask_data = []

# #     # Iterar sobre los archivos .h5 en el directorio
# #     for file_name in os.listdir(directory_path):
# #         if file_name.endswith(".h5"):
# #             file_path = os.path.join(directory_path, file_name)

# #             # Cargar imágenes y máscaras
# #             images = load_h5_file(file_path, dataset_name=image_dataset_name)
# #             masks = load_h5_file(file_path, dataset_name=mask_dataset_name)

# #             # Agregar a las listas
# #             image_data.append(images)
# #             mask_data.append(masks)

#     # # Convertir listas a arrays de NumPy
#     # image_data = np.concatenate(image_data, axis=0)
#     # mask_data = np.concatenate(mask_data, axis=0)

#     # return image_data, mask_data

# # Cargar datos
# # images, masks = load_data(directory, image_dataset_name="images", mask_dataset_name="masks")
# images, masks = load_data(directory)

# print("Imágenes cargadas:", images.shape)
# print("Máscaras cargadas:", masks.shape)


# # # Cargar los archivos en un array
# # h5_files = [f for f in os.listdir(directory) if f.endswith('.h5')]
# # print(f"Found {len(h5_files)} .h5 files:\nExample file names:{h5_files[:3]}")

In [131]:
# # Generar Dataset
# def data_generator(h5_paths, batch_size):
#     def generator():
#         for file_path in h5_paths:
#             images, masks = load_h5_data(file_path)
#             images = normalize_image(images) # Normalizar imágenes
#             masks = np.stack([process_mask(mask) for mask in masks], axis=0) # Preprocesar máscaras
#             for i in range(images.shape[0]): # Iterar por muestras
#                 yield images[i], masks[i]

#     dataset = tf.data.Dataset.from_generator(
#         generator,
#         output_signature=(
#             tf.TensorSpec(shape=(None, None, 4), dtype=tf.float32), # Imágenes
#             tf.TensorSpec(shape=(None, None, 3), dtype=tf.int32) # Máscaras
#         )
#     )
#     return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [159]:
def process_h5_file(h5_path, image_dataset_name, mask_dataset_name):
    with h5py.File(h5_path, 'r') as f:
        images = f[image_dataset_name][:]
        masks = f[mask_dataset_name][:]
        
    return images, masks

def generate_data_from_h5(directory, image_dataset_name="images", mask_dataset_name="masks"):
    h5_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.h5')]
    print(f"Found {len(h5_files)} .h5 files:\nExample file names: {h5_files[:3]}")
    
    for h5_file in h5_files:
        images, masks = process_h5_file(h5_file, image_dataset_name, mask_dataset_name)
        for img, mask in zip(images, masks):
            yield img, mask  # Retorna una imagen y su máscara

In [160]:
# Crear el generador
data_generator = generate_data_from_h5(data_dir, image_dataset_name="images", mask_dataset_name="masks")

# Convertir a un dataset de TensorFlow
import tensorflow as tf

def tf_data_generator():
    for img, mask in data_generator:
        # Convertir imágenes y máscaras a tensores
        yield tf.convert_to_tensor(img, dtype=tf.float32), tf.convert_to_tensor(mask, dtype=tf.float32)

# Crear el tf.data.Dataset
tf_dataset = tf.data.Dataset.from_generator(
    tf_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(240, 240, 4), dtype=tf.float32),  # Cambiar la forma según el dataset
        tf.TensorSpec(shape=(240, 240, 3), dtype=tf.float32)
    )
)

# Agregar procesamiento adicional
batch_size = 16
tf_dataset = (
    tf_dataset
    .shuffle(buffer_size=100)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
# Obtener la lista de archivos .h5
h5_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.h5')]

# Dividir en entrenamiento y validación
train_files, val_files = train_test_split(h5_files, test_size=0.2, random_state=42)

# Función para leer un archivo .h5 y devolver imágenes y máscaras
def process_h5_file(file_path, image_dataset_name="images", mask_dataset_name="masks"):
    with h5py.File(file_path.numpy().decode("utf-8"), "r") as f:
        images = f[image_dataset_name][:]
        masks = f[mask_dataset_name][:]
    return images, masks

# Función envolvente para que sea compatible con tf.data.Dataset
def tf_process_h5_file(file_path):
    images, masks = tf.py_function(
        func=process_h5_file,
        inp=[file_path],
        Tout=(tf.float32, tf.float32)
    )
    # Ajusta las dimensiones según tus necesidades
    images.set_shape([None, 240, 240, 4])  # (número de imágenes, altura, ancho, canales)
    masks.set_shape([None, 240, 240, 3])   # (número de máscaras, altura, ancho, clases)
    return images, masks

# Crear un dataset a partir de la lista de archivos
def create_dataset(file_list, batch_size=16, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices(file_list)
    dataset = dataset.map(tf_process_h5_file, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.flat_map(lambda images, masks: tf.data.Dataset.from_tensor_slices((images, masks)))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=100)
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
# Crear datasets de entrenamiento y validación
batch_size = 16

train_dataset = create_dataset(train_files, batch_size=batch_size, shuffle=True)
val_dataset = create_dataset(val_files, batch_size=batch_size, shuffle=False)

# Verificar el dataset
for images, masks in train_dataset.take(1):
    print(f"Imágenes: {images.shape}, Máscaras: {masks.shape}")

In [132]:
train_files, val_files = train_test_split(directory, test_size=0.2, random_state=42)

# Crear datasets
batch_size = 8
train_dataset = data_generator(train_files, batch_size)
val_dataset = data_generator(val_files, batch_size)

In [133]:
for batch_images, batch_masks in train_dataset.take(1):
    print("Shape of input images:", batch_images.shape)
    print("Shape of masks:", batch_masks.shape)
    print("Shape of model output:", model(batch_images).shape)


UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'n', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):

  File "C:\Users\taylo\anaconda3\envs\py_31016\lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    ret = func(*args)

  File "C:\Users\taylo\anaconda3\envs\py_31016\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\taylo\anaconda3\envs\py_31016\lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\taylo\AppData\Local\Temp\ipykernel_16440\2193334555.py", line 5, in generator
    images, masks = load_h5_data(file_path)

  F

In [51]:
# # Construir dirección para los .h5 desde el directorio
# h5_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.h5')]

# np.random.seed(42)
# np.random.shuffle(h5_files)

# # Dividir la base de datos entre entrenamiento y validación (80:10)
# split_idx = int(0.8 * len(h5_files))
# train_files = h5_files[:split_idx]
# val_files = h5_files[split_idx:]

# # Crear los datasets de entrenamiento y validación
# train_dataset = create_brain_scan_dataset(train_files)
# train_dataset = train_dataset.shuffle(buffer_size=100).batch(32).prefetch(tf.data.AUTOTUNE)

# val_dataset = create_brain_scan_dataset(val_files)
# val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# # Visualizar un ejemplo del dataset
# for images, masks in train_dataset.take(1):
#     print("Train Dataset")
#     print("Tamaño de las imágenes:", images.shape)
#     print("Tamaño de las máscaras:", masks.shape)

# # Verificar val_dataset
# for images, masks in val_dataset.take(1):
#     print("\nValidation Dataset")
#     print("Tamaño de las imágenes:", images.shape)
#     print("Tamaño de las máscaras:", masks.shape)

Train Dataset
Tamaño de las imágenes: (32, 240, 240, 4)
Tamaño de las máscaras: (32, 240, 240, 3)

Validation Dataset
Tamaño de las imágenes: (32, 240, 240, 4)
Tamaño de las máscaras: (32, 240, 240, 3)


In [134]:
# Bloque SE-ResNet
def se_resnet_block(input_tensor, n_filters, kernel_size=3, stride=1, reduction_ratio=16):
    # Bloque Residual
    # Primera convolución
    x = layers.Conv2D(n_filters, kernel_size, strides=stride, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Segunda convolución
    x = layers.Conv2D(n_filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Bloque SE
    # Squeeze - Global Average Pooling
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Dense(n_filters // reduction_ratio, activation='relu')(se) # Bottleneck
    se = layers.Dense(n_filters, activation='sigmoid')(se) # Excitación
    se = layers.Reshape((1, 1 , n_filters))(se) # Ajuste de dimensiones
    x = layers.Multiply()([x, se]) # Recalibración

    # Shortcut connection - Identity
    # shortcut = input_tensor  # Identidad
    if input_tensor.shape[-1] != n_filters:
        shortcut = layers.Conv2D(n_filters, (1, 1), padding='same')(input_tensor)
    else:
        shortcut = input_tensor

    x = layers.Add()([x, shortcut])  # Suma residual
    x = layers.ReLU()(x)

    return x

In [135]:
# Attention Gate
def attention_gate(dec, enc, n_filters):
    # Reducir dimensión de Encoder
    enc1 = layers.Conv2D(n_filters, (1, 1), padding='same')(enc)
    enc1 = layers.BatchNormalization()(enc1)

    # Reducir dimensión de Gatting Signal
    gatting_sig = layers.Conv2D(n_filters, (1, 1), padding='same')(dec)
    gatting_sig = layers.BatchNormalization()(gatting_sig)

    # Combinar Gatting Signal y Skip Connection
    combined = layers.Add()([enc1, gatting_sig])
    combined = layers.ReLU()(combined)

    # Mapa de Atención
    attention_map = layers.Conv2D(1, (1, 1), activation='sigmoid',padding='same')(combined)

    # Aplicar mapa de Atención
    output = layers.Multiply()([enc, attention_map])

    return output

In [136]:
# Encoder
def encoder_block(input_tensor, n_filters, apply_pooling=True):
    x = se_resnet_block(input_tensor, n_filters)
    p = layers.MaxPooling2D((2, 2))(x)

    return x, p # x se utiliza en el skip connection; p pasa al siguiente bloque
    
# Downsampling
def build_encoder(input_tensor, filters_list):
    skips = []
    
    # Primera Convolución
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(input_tensor)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    for i, n_filters in enumerate(filters_list):
        # Verifica si es el último bloque
        apply_pooling = i < len(filters_list) - 1  # True excepto en el último bloque
        x, p = encoder_block(x, n_filters, apply_pooling=apply_pooling)
        skips.append(x)  # Guardar skip connection
        
        # Avanzar al siguiente bloque solo si hay pooling
        if apply_pooling:
            x = p

    return x, skips

In [137]:
# Decoder
def decoder_block(input_tensor, skip_tensor, n_filters):
    # Gatting Signal
    gated_skip = attention_gate(input_tensor, skip_tensor, n_filters)
    gated_skip = layers.Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(gated_skip)

    x = layers.Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor) # Upsampling
    x = layers.Concatenate()([x, gated_skip])  # Combina con skip connection
    x = se_resnet_block(x, n_filters)

    return x

# Upsampling
def build_decoder(encoder_output, skips, filters_list):
    x = encoder_output
    for filters, skip_tensor in zip(filters_list, reversed(skips)):        
        x = decoder_block(x, skip_tensor, filters)

    x = layers.Conv2D(3, (1, 1), strides=1, padding='same', activation='softmax')(x)
    
    return x

In [138]:
# SE-ResANet
def se_resanet(input_shape, encoder_filters, decoder_filters):
    input_tensor = layers.Input(shape=input_shape)

    # Encoder
    encoder_output, skips = build_encoder(input_tensor, encoder_filters)

    # Decoder
    decoder_output = build_decoder(encoder_output, skips, decoder_filters)

    # Output
    # outputs = layers.Conv2D(filters=4, kernel_size=1, strides=1, padding='same', activation='softmax')(decoder_output)
    outputs = layers.Conv2D(4, (1, 1), activation='softmax')(decoder_output)

    # Modelo
    return models.Model(inputs=input_tensor, outputs=outputs)

In [139]:
input_shape = (240, 240, 4)  # Dimensiones de las imágenes BraTS2020

encoder_filters = [64, 128, 256, 512] # Filtros de cada nivel del Encoder
decoder_filters = [512, 256, 128, 64] # Filtros de cada nivel del Decoder

model = se_resanet(input_shape, encoder_filters, decoder_filters)

# Resumen
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_309 (Conv2D) │ (None, 240, 240,  │      1,184 │ input_layer_8[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_168 (ReLU)    │ (None, 240, 240,  │          0 │ conv2d_309[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_40    │ (None, 120, 120,  │          0 │ re_lu_168[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_310 (Conv2D) │ (None, 120, 120,  │     18,496 │ max_pooling2d_40… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │        256 │ conv2d_310[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_169 (ReLU)    │ (None, 120, 120,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_311 (Conv2D) │ (None, 120, 120,  │     36,928 │ re_lu_169[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │        256 │ conv2d_311[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_128 (Dense)   │ (None, 4)         │        260 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_129 (Dense)   │ (None, 64)        │        320 │ dense_128[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_64          │ (None, 1, 1, 64)  │          0 │ dense_129[0][0]   │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_96         │ (None, 120, 120,  │          0 │ batch_normalizat… │
│ (Multiply)          │ 64)               │            │ reshape_64[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_312 (Conv2D) │ (None, 120, 120,  │      2,112 │ max_pooling2d_40… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_96 (Add)        │ (None, 120, 120,  │          0 │ multiply_96[0][0… │
│                     │ 64)               │            │ conv2d_312[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_170 (ReLU)    │ (None, 120, 120,  │          0 │ add_96[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 19,006,639 (72.50 MB)

 Trainable params: 18,995,119 (72.46 MB)

 Non-trainable params: 11,520 (45.00 KB)

In [140]:
# Dice Coefficient
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = backend.flatten(y_true)
    y_pred_f = backend.flatten(y_pred)
    intersection = backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (backend.sum(y_true_f) + backend.sum(y_pred_f) + smooth)

# Recall
def recall(y_true, y_pred):
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = backend.sum(backend.round(backend.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + backend.epsilon())

In [141]:
# Compilar
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Optimizar con Adam
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', dice_coefficient, recall]  # Métrica de precisión
)

In [73]:
# # Dataset con preprocesamiento
# train_dataset = train_dataset.map(
#     lambda img, mask: (
#         tf.ensure_shape(img, [None, 240, 240, 4]), 
#         tf.ensure_shape(mask, [None, 240, 240, 3])  # O convertir máscara a etiquetas aquí
#     ),
#     num_parallel_calls=tf.data.AUTOTUNE
# )

In [74]:
# # Entrenar el modelo
# train_steps = tf.data.experimental.cardinality(train_dataset).numpy()
# val_steps = tf.data.experimental.cardinality(val_dataset).numpy()

# print("Train steps:", train_steps)
# print("Validation steps:", val_steps)

# model.fit(
#     train_dataset,
#     epochs=50,
#     validation_data=val_dataset,
#     # steps_per_epoch=train_steps,
#     # validation_steps=val_steps
# )

In [142]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50
)

Epoch 1/50


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, None, None, 3), output.shape=(None, None, None, 4)

In [14]:
# Evaluar el modelo
val_loss, val_accuracy, val_dice, val_recall = model.evaluate(val_dataset)

print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")
print(f"Validation Dice Coefficient: {val_dice}")
print(f"Validation Recall: {val_recall}")

ValueError: as_list() is not defined on an unknown TensorShape.